In [1]:
import pandas as pd

In [2]:
energy_values = pd.read_csv("../data/all_hourly_energy_values.csv")
energy_values

,Unnamed: 0,hour,production,imports,exports,consumption,year,month,day
0,3,01:00,29912.990,4.00,368.20,29548.79,2021,1,1
1,4,02:00,28470.540,3.00,349.00,28124.54,2021,1,1
2,5,03:00,26727.150,91.00,233.00,26585.15,2021,1,1
3,6,04:00,25528.950,96.00,241.00,25383.95,2021,1,1
4,7,05:00,24913.100,76.00,213.00,24776.10,2021,1,1
...,...,...,...,...,...,...,...,...,...
27835,22,20:00,40822.393,279.75,359.66,40742.48,2024,3,12
27836,23,21:00,41953.750,292.82,326.28,41920.29,2024,3,12
27837,24,22:00,40657.890,281.29,345.87,40593.31,2024,3,12
27838,25,23:00,39384.410,310.59,310.26,39384.74,2024,3,12


In [3]:
weather = pd.read_csv("../data/noaa-measurements.csv")
weather['STATION'] = weather['STATION'].astype("str")

/var/folders/jb/4xb8tv1j2bn4xbb3c9f734br0000gn/T/ipykernel_33374/4235820603.py:1: DtypeWarning: Columns (2,6,7,8,9,10,11,12,13,14,15,16,17,18,22,23,26,27,30,31,32,33,35,36,38,39,40,41,42,43,48) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv("../data/noaa-measurements.csv")


In [4]:

weather_clean = weather[weather['STATION'].str.startswith('17')]  # Filter out the ones from outside Turkey
weather_clean = weather_clean.dropna(axis='columns', thresh=len(weather_clean) * 0.95)


In [5]:
weather_clean['DATE'] = pd.to_datetime(weather_clean['DATE'])
useful_columns = ['STATION', 'DATE', 'CIG', 'DEW', 'MA1', 'SLP', 'TMP',
       'VIS', 'WND']
weather_clean = weather_clean[useful_columns][weather_clean['DATE'].dt.minute == 0]

In [6]:
weather_clean

,STATION,DATE,CIG,DEW,MA1,SLP,TMP,VIS,WND
0,17192099999,2020-01-01 06:00:00,"00900,1,9,N","+0006,1","99999,9,09066,1","10193,1","+0019,1","004000,1,9,9","250,1,N,0005,1"
1,17192099999,2020-01-01 07:00:00,"99999,9,9,N","+0005,1","99999,9,09070,1","10198,1","+0019,1","005000,1,9,9","250,1,N,0010,1"
2,17192099999,2020-01-01 08:00:00,"99999,9,9,N","+0004,1","99999,9,09073,1","10202,1","+0019,1","003000,1,9,9","310,1,N,0010,1"
3,17192099999,2020-01-01 09:00:00,"00900,1,9,N","+0006,1","99999,9,09071,1","10200,1","+0024,1","005000,1,9,9","280,1,N,0005,1"
4,17192099999,2020-01-01 10:00:00,"99999,9,9,N","+0007,1","99999,9,09066,1","10194,1","+0026,1","005000,1,9,9","290,1,N,0010,1"
...,...,...,...,...,...,...,...,...,...
6238845,17060099999,2024-03-27 09:00:00,"03000,1,C,N","+0077,1","99999,9,10098,1","10132,1","+0158,1","010000,1,9,9","110,1,N,0031,1"
6238852,17060099999,2024-03-27 12:00:00,"03000,1,C,9","+0058,1","99999,9,10087,1","10121,1","+0196,1","010000,1,9,9","080,1,N,0031,1"
6238859,17060099999,2024-03-27 15:00:00,"06000,1,9,9","+0072,1","99999,9,10082,1","10116,1","+0201,1","010000,1,9,9","090,1,N,0026,1"
6238866,17060099999,2024-04-01 18:00:00,"06000,1,9,9","+0109,1","99999,9,10129,1","10163,1","+0166,1","010000,1,9,9","999,9,C,0000,1"


In [7]:
comma_split = weather_clean.copy()

for col in ['CIG', 'DEW', 'MA1', 'SLP', 'TMP', 'VIS', 'WND']:
    max_cols = int(comma_split[col].str.split(',').str.len().max())
    column_names = [col + str(i) for i in range(1, max_cols + 1)]
    comma_split[column_names] = comma_split[col].str.split(',', expand=True)


In [8]:
comma_split= comma_split.drop(['CIG', 'DEW', 'MA1', 'SLP', 'TMP', 'VIS', 'WND'],axis = 1)
comma_split

,STATION,DATE,CIG1,CIG2,CIG3,CIG4,DEW1,DEW2,MA11,MA12,...,TMP2,VIS1,VIS2,VIS3,VIS4,WND1,WND2,WND3,WND4,WND5
0,17192099999,2020-01-01 06:00:00,00900,1,9,N,+0006,1,99999,9,...,1,004000,1,9,9,250,1,N,0005,1
1,17192099999,2020-01-01 07:00:00,99999,9,9,N,+0005,1,99999,9,...,1,005000,1,9,9,250,1,N,0010,1
2,17192099999,2020-01-01 08:00:00,99999,9,9,N,+0004,1,99999,9,...,1,003000,1,9,9,310,1,N,0010,1
3,17192099999,2020-01-01 09:00:00,00900,1,9,N,+0006,1,99999,9,...,1,005000,1,9,9,280,1,N,0005,1
4,17192099999,2020-01-01 10:00:00,99999,9,9,N,+0007,1,99999,9,...,1,005000,1,9,9,290,1,N,0010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6238845,17060099999,2024-03-27 09:00:00,03000,1,C,N,+0077,1,99999,9,...,1,010000,1,9,9,110,1,N,0031,1
6238852,17060099999,2024-03-27 12:00:00,03000,1,C,9,+0058,1,99999,9,...,1,010000,1,9,9,080,1,N,0031,1
6238859,17060099999,2024-03-27 15:00:00,06000,1,9,9,+0072,1,99999,9,...,1,010000,1,9,9,090,1,N,0026,1
6238866,17060099999,2024-04-01 18:00:00,06000,1,9,9,+0109,1,99999,9,...,1,010000,1,9,9,999,9,C,0000,1


In [9]:
selected_features = {'CIG1': "sky_condition", 'DEW1': "dewpoint_temperature", 'MA13': "station_pressure", 
                     'SLP1': "sea_level_pressure", 'TMP1': "air_temperature", 'VIS1': "visibility",
        'WND1': "wind_angle", 'WND4': "wind_speed"}

In [10]:
value_columns = ['CIG1', 'CIG2', 'CIG3', 'CIG4', 'DEW1', 'DEW2',
       'MA11', 'MA12', 'MA13', 'MA14', 'SLP1', 'SLP2', 'TMP1', 'TMP2', 'VIS1',
       'VIS2', 'VIS3', 'VIS4', 'WND1', 'WND2', 'WND3', 'WND4', 'WND5']
pivoted = comma_split.pivot_table(index=['DATE'], columns='STATION', values=selected_features.keys(), aggfunc="first").rename(columns=selected_features)
pivoted

sky_condition                                      \
STATION               17020099999 17022099999 17022399999 17024099999   
DATE                                                                    
2020-01-01 00:00:00         03000       00750         NaN         NaN   
2020-01-01 01:00:00         99999       99999         NaN         NaN   
2020-01-01 02:00:00         99999       99999         NaN         NaN   
2020-01-01 03:00:00         03000       00900         NaN         NaN   
2020-01-01 04:00:00         99999       99999         NaN         NaN   
...                           ...         ...         ...         ...   
2024-04-01 19:00:00           NaN         NaN         NaN         NaN   
2024-04-01 20:00:00           NaN         NaN         NaN         NaN   
2024-04-01 21:00:00           NaN         NaN         NaN         NaN   
2024-04-01 22:00:00           NaN         NaN         NaN         NaN   
2024-04-01 23:00:00           NaN         NaN         NaN         NaN   

                                                                     \
STATION             17026099999 17030099999 17031099999 17033099999   
DATE                                                                  
2020-01-01 00:00:00         NaN       00840       00840         NaN   
2020-01-01 01:00:00         NaN       99999         NaN         NaN   
2020-01-01 02:00:00         NaN       99999         NaN         NaN   
2020-01-01 03:00:00         NaN       03000       00840         NaN   
2020-01-01 04:00:00         NaN       99999         NaN         NaN   
...                         ...         ...         ...         ...   
2024-04-01 19:00:00         NaN       99999         NaN         NaN   
2024-04-01 20:00:00         NaN       99999         NaN         NaN   
2024-04-01 21:00:00       22000       99999       99999       99999   
2024-04-01 22:00:00         NaN       99999         NaN         NaN   
2024-04-01 23:00:00         NaN       99999         NaN         NaN   

                                             ...  wind_speed              \
STATION             17034099999 17038099999  ... 17320099999 17330099999   
DATE                                         ...                           
2020-01-01 00:00:00       02400       00120  ...         NaN         NaN   
2020-01-01 01:00:00       99999         NaN  ...         NaN         NaN   
2020-01-01 02:00:00       99999         NaN  ...         NaN         NaN   
2020-01-01 03:00:00       02400       00120  ...         NaN         NaN   
2020-01-01 04:00:00       99999         NaN  ...         NaN         NaN   
...                         ...         ...  ...         ...         ...   
2024-04-01 19:00:00       99999         NaN  ...         NaN         NaN   
2024-04-01 20:00:00       99999         NaN  ...         NaN         NaN   
2024-04-01 21:00:00       99999       99999  ...        0005         NaN   
2024-04-01 22:00:00       99999         NaN  ...         NaN         NaN   
2024-04-01 23:00:00       99999         NaN  ...         NaN         NaN   

                                                                     \
STATION             17340099999 17350033308 17352099999 17355099999   
DATE                                                                  
2020-01-01 00:00:00        0000        0015        0015        0000   
2020-01-01 01:00:00        0000         NaN         NaN         NaN   
2020-01-01 02:00:00        0000         NaN         NaN         NaN   
2020-01-01 03:00:00        0000        0021        0026        0000   
2020-01-01 04:00:00        0005         NaN         NaN         NaN   
...                         ...         ...         ...         ...   
2024-04-01 19:00:00        0000         NaN         NaN         NaN   
2024-04-01 20:00:00        0000         NaN         NaN         NaN   
2024-04-01 21:00:00        0000        0036        0015        0005   
2024-04-01 22:00:00        0000         NaN         NaN         NaN   
2024-04-01 23:

In [11]:
dropped = pivoted.dropna(axis='columns', thresh=len(pivoted) * 0.95)
dropped

sky_condition                                      \
STATION               17030099999 17064099999 17066199999 17095099999   
DATE                                                                    
2020-01-01 00:00:00         00840       01050       00750       22000   
2020-01-01 01:00:00         99999       01000       99999       99999   
2020-01-01 02:00:00         99999       01000       22000       99999   
2020-01-01 03:00:00         03000       01050       22000       02400   
2020-01-01 04:00:00         99999       00600       22000       22000   
...                           ...         ...         ...         ...   
2024-04-01 19:00:00         99999       99999       99999       99999   
2024-04-01 20:00:00         99999       99999       99999       99999   
2024-04-01 21:00:00         99999       06000       99999       99999   
2024-04-01 22:00:00         99999       99999       99999       99999   
2024-04-01 23:00:00         99999       99999       99999       99999   

                                                                     \
STATION             17130099999 17190099999 17240099999 17302099999   
DATE                                                                  
2020-01-01 00:00:00       03000       03000       22000       22000   
2020-01-01 01:00:00       99999       99999       22000       22000   
2020-01-01 02:00:00       99999       99999       22000       22000   
2020-01-01 03:00:00       03000       03000       22000       22000   
2020-01-01 04:00:00       99999       99999       99999       22000   
...                         ...         ...         ...         ...   
2024-04-01 19:00:00       99999       99999       99999       99999   
2024-04-01 20:00:00       99999       99999       99999       99999   
2024-04-01 21:00:00       99999       99999       99999       99999   
2024-04-01 22:00:00       99999       99999       99999       99999   
2024-04-01 23:00:00       99999       99999       99999       99999   

                    dewpoint_temperature              ...  wind_angle  \
STATION                      17030099999 17064099999  ... 17240099999   
DATE                                                  ...               
2020-01-01 00:00:00                +0076       +0035  ...         330   
2020-01-01 01:00:00                +0078       +0033  ...         270   
2020-01-01 02:00:00                +0075       +0032  ...         360   
2020-01-01 03:00:00                +0074       +0019  ...         340   
2020-01-01 04:00:00                +0075       -0012  ...         310   
...                                  ...         ...  ...         ...   
2024-04-01 19:00:00                +0089       +0109  ...         230   
2024-04-01 20:00:00                +0106       +0104  ...         120   
2024-04-01 21:00:00                +0111       +0102  ...         120   
2024-04-01 22:00:00                +0096       +0102  ...         340   
2024-04-01 23:00:00                +0100       +0100  ...         999   

                                 wind_speed                          \
STATION             17302099999 17030099999 17064099999 17066199999   
DATE                                                                  
2020-01-01 00:00:00         340        0000        0021        0010   
2020-01-01 01:00:00         350        0000        0010        0005   
2020-01-01 02:00:00         340        0000        0026        0005   
2020-01-01 03:00:00         340        0000        0021        0010   
2020-01-01 04:00:00         350        0000        0005        0005   
...                         ...         ...         ...         ...   
2024-04-01 19:00:00         999        0005        0000        0005   
2024-04-01 20:00:00         340        0010        0000        0005   
2024-04-01 21:00:00         999        0000        0000        0000   
2024-04-01 22:00:00         999        0000        0000        0000   
2024-04-01 23:00:00         999        0000        0000 

In [12]:
station_names = {"17030099999": "Samsun",
            "17064099999": "Kartal",
            "17066199999": "İzmit",
            "17095099999": "Erzurum",
            "17130099999": "Ankara",
            "17190099999": "Afyon",
            "17240099999": "Isparta",
            "17302099999": "Antalya"}

dropped = dropped.rename(columns=station_names)
dropped

sky_condition                                              \
STATION                    Samsun Kartal  İzmit Erzurum Ankara  Afyon Isparta   
DATE                                                                            
2020-01-01 00:00:00         00840  01050  00750   22000  03000  03000   22000   
2020-01-01 01:00:00         99999  01000  99999   99999  99999  99999   22000   
2020-01-01 02:00:00         99999  01000  22000   99999  99999  99999   22000   
2020-01-01 03:00:00         03000  01050  22000   02400  03000  03000   22000   
2020-01-01 04:00:00         99999  00600  22000   22000  99999  99999   99999   
...                           ...    ...    ...     ...    ...    ...     ...   
2024-04-01 19:00:00         99999  99999  99999   99999  99999  99999   99999   
2024-04-01 20:00:00         99999  99999  99999   99999  99999  99999   99999   
2024-04-01 21:00:00         99999  06000  99999   99999  99999  99999   99999   
2024-04-01 22:00:00         99999  99999  99999   99999  99999  99999   99999   
2024-04-01 23:00:00         99999  99999  99999   99999  99999  99999   99999   

                            dewpoint_temperature         ... wind_angle  \
STATION             Antalya               Samsun Kartal  ...    Isparta   
DATE                                                     ...              
2020-01-01 00:00:00   22000                +0076  +0035  ...        330   
2020-01-01 01:00:00   22000                +0078  +0033  ...        270   
2020-01-01 02:00:00   22000                +0075  +0032  ...        360   
2020-01-01 03:00:00   22000                +0074  +0019  ...        340   
2020-01-01 04:00:00   22000                +0075  -0012  ...        310   
...                     ...                  ...    ...  ...        ...   
2024-04-01 19:00:00   99999                +0089  +0109  ...        230   
2024-04-01 20:00:00   99999                +0106  +0104  ...        120   
2024-04-01 21:00:00   99999                +0111  +0102  ...        120   
2024-04-01 22:00:00   99999                +0096  +0102  ...        340   
2024-04-01 23:00:00   99999                +0100  +0100  ...        999   

                            wind_speed                                    \
STATION             Antalya     Samsun Kartal İzmit Erzurum Ankara Afyon   
DATE                                                                       
2020-01-01 00:00:00     340       0000   0021  0010    0005   0005  0005   
2020-01-01 01:00:00     350       0000   0010  0005    0000   0005  0005   
2020-01-01 02:00:00     340       0000   0026  0005    0000   0005  0010   
2020-01-01 03:00:00     340       0000   0021  0010    0005   0005  0005   
2020-01-01 04:00:00     350       0000   0005  0005    0005   0005  0010   
...                     ...        ...    ...   ...     ...    ...   ...   
2024-04-01 19:00:00     999       0005   0000  0005    0000   0000  0005   
2024-04-01 20:00:00     340       0010   0000  0005    0000   0005  0000   
2024-04-01 21:00:00     999       0000   0000  0000    0000   0005  0010   
2024-04-01 22:00:00     999       0000   0000  0000    0000   0015  0000   
2024-04-01 23:00:00     999       0000   0000  0000    0000   0005  0000   

                                     
STATION             Isparta Antalya  
DATE                                 
2020-01-01 00:00:00    0005    0015  
2020-01-01 01:00:00    0005    0015  
2020-01-01 02:00:00    0000    0015  
2020-01-01 03:00:00    0005    0010  
2020-01-01 04:00:00    0005    0015  
...                     ...     ...  
2024-04-01 19:00:00    0005    0000  
2024-04-01 20:00:00    0005    0005  
2024-04-01 21:00:00    0005    0000  
2024-04-01 22:00:00    0005    0000  
2024-04-01 23:00:00    0000    0000  

[36811 rows x 64 columns]

In [13]:
#dropped.columns = ['_'.join(str(col)).strip() for col in dropped.columns.values]
dropped.columns = dropped.columns.map('|'.join).str.strip('|')

In [14]:
import numpy as np 


ratios = dropped.replace("999", np.nan).replace("9999", np.nan).replace("99999", np.nan).replace("999999", np.nan).isna().sum() / len(dropped)

print(ratios.to_string())

/var/folders/jb/4xb8tv1j2bn4xbb3c9f734br0000gn/T/ipykernel_33374/3665081765.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratios = dropped.replace("999", np.nan).replace("9999", np.nan).replace("99999", np.nan).replace("999999", np.nan).isna().sum() / len(dropped)


sky_condition|Samsun            0.488984
sky_condition|Kartal            0.493657
sky_condition|İzmit             0.575127
sky_condition|Erzurum           0.499905
sky_condition|Ankara            0.535492
sky_condition|Afyon             0.574883
sky_condition|Isparta           0.492054
sky_condition|Antalya           0.477846
dewpoint_temperature|Samsun     0.005080
dewpoint_temperature|Kartal     0.021352
dewpoint_temperature|İzmit      0.028171
dewpoint_temperature|Erzurum    0.006737
dewpoint_temperature|Ankara     0.037597
dewpoint_temperature|Afyon      0.011654
dewpoint_temperature|Isparta    0.030860
dewpoint_temperature|Antalya    0.031974
station_pressure|Samsun         0.005189
station_pressure|Kartal         0.021407
station_pressure|İzmit          0.028252
station_pressure|Erzurum        0.007172
station_pressure|Ankara         0.037679
station_pressure|Afyon          0.011681
station_pressure|Isparta        0.030915
station_pressure|Antalya        0.032028
sea_level_pressu

In [15]:
energy_values["hour"] = energy_values["hour"].str[:2].astype(int)
energy_values['DATE'] = pd.to_datetime(energy_values[['year', 'month', 'day', 'hour']])
result_df = pd.merge(energy_values, dropped, on='DATE', how='left')

# Handling Missing Values

In [16]:


for col in result_df.columns:
    if "sky_condition" in col:
        present_column_name = col + "|" + "Present"
        result_df[present_column_name] = ~result_df[col].isna()
result_df

,Unnamed: 0,hour,production,imports,exports,consumption,year,month,day,DATE,...,wind_speed|Isparta,wind_speed|Antalya,sky_condition|Samsun|Present,sky_condition|Kartal|Present,sky_condition|İzmit|Present,sky_condition|Erzurum|Present,sky_condition|Ankara|Present,sky_condition|Afyon|Present,sky_condition|Isparta|Present,sky_condition|Antalya|Present
0,3,1,29912.990,4.00,368.20,29548.79,2021,1,1,2021-01-01 01:00:00,...,0000,0010,True,True,True,True,True,True,True,True
1,4,2,28470.540,3.00,349.00,28124.54,2021,1,1,2021-01-01 02:00:00,...,0000,0010,True,True,True,True,True,True,True,True
2,5,3,26727.150,91.00,233.00,26585.15,2021,1,1,2021-01-01 03:00:00,...,0000,0010,True,True,True,True,True,True,True,True
3,6,4,25528.950,96.00,241.00,25383.95,2021,1,1,2021-01-01 04:00:00,...,0000,0005,True,True,True,True,True,True,True,True
4,7,5,24913.100,76.00,213.00,24776.10,2021,1,1,2021-01-01 05:00:00,...,0000,0010,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27835,22,20,40822.393,279.75,359.66,40742.48,2024,3,12,2024-03-12 20:00:00,...,0041,0000,True,True,True,True,True,True,True,True
27836,23,21,41953.750,292.82,326.28,41920.29,2024,3,12,2024-03-12 21:00:00,...,NaN,NaN,False,False,False,False,False,False,False,False
27837,24,22,40657.890,281.29,345.87,40593.31,2024,3,12,2024-03-12 22:00:00,...,0036,0000,True,True,True,True,True,True,True,True
27838,25,23,39384.410,310.59,310.26,39384.74,2024,3,12,2024-03-12 23:00:00,...,0051,0000,True,True,True,True,True,True,True,True


In [17]:
! pip install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [18]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

In [19]:
ii = IterativeImputer()
without_date = result_df.drop(["DATE", "sea_level_pressure|Afyon", "sea_level_pressure|Erzurum", "Unnamed: 0"], axis=1).replace("999", np.nan).replace("9999", np.nan).replace("99999", np.nan).replace("999999", np.nan)


In [20]:
without_date.columns

Index(['hour', 'production', 'imports', 'exports', 'consumption', 'year',
       'month', 'day', 'sky_condition|Samsun', 'sky_condition|Kartal',
       'sky_condition|İzmit', 'sky_condition|Erzurum', 'sky_condition|Ankara',
       'sky_condition|Afyon', 'sky_condition|Isparta', 'sky_condition|Antalya',
       'dewpoint_temperature|Samsun', 'dewpoint_temperature|Kartal',
       'dewpoint_temperature|İzmit', 'dewpoint_temperature|Erzurum',
       'dewpoint_temperature|Ankara', 'dewpoint_temperature|Afyon',
       'dewpoint_temperature|Isparta', 'dewpoint_temperature|Antalya',
       'station_pressure|Samsun', 'station_pressure|Kartal',
       'station_pressure|İzmit', 'station_pressure|Erzurum',
       'station_pressure|Ankara', 'station_pressure|Afyon',
       'station_pressure|Isparta', 'station_pressure|Antalya',
       'sea_level_pressure|Samsun', 'sea_level_pressure|Kartal',
       'sea_level_pressure|İzmit', 'sea_level_pressure|Ankara',
       'sea_level_pressure|Isparta', 'sea_lev

In [21]:
array = without_date.to_numpy()
array.shape

(27840, 78)

In [22]:
imputed = ii.fit_transform(array)

KeyboardInterrupt: 

In [ ]:
without_date.columns

In [ ]:
imputed

In [ ]:
imputed_df = pd.DataFrame(imputed, columns = without_date.columns)
imputed_df

In [ ]:
imputed_df

# Save the file

In [ ]:
imputed_df.to_csv("imputed.csv")